In [27]:
import cv2
import os
import numpy as np
from shutil import copyfile
import xml.etree.cElementTree as ET

def newXMLPASCALfile(height, width, path,filename):

    annotation = ET.Element("annotation", verified="yes")
    ET.SubElement(annotation, "folder").text = "images"
    ET.SubElement(annotation, "filename").text = filename.replace(".ppm",".jpg")
    ET.SubElement(annotation, "path").text = path

    source = ET.SubElement(annotation, "source")
    ET.SubElement(source, "database").text = "test"

    size = ET.SubElement(annotation, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"

    ET.SubElement(annotation, "segmented").text = "0"
    tree = ET.ElementTree(annotation)
    
    return tree

def appendXMLPASCAL(curr_et_object,x1, y1, x2, y2, filename):
    et_object = ET.SubElement(curr_et_object.getroot(), "object")
    ET.SubElement(et_object, "name").text = "sign"
    
    bndbox = ET.SubElement(et_object, "bndbox")
    ET.SubElement(bndbox, "xmin").text = str(x1)
    ET.SubElement(bndbox, "ymin").text = str(y1)
    ET.SubElement(bndbox, "xmax").text = str(x2)
    ET.SubElement(bndbox, "ymax").text = str(y2)
    filename = filename.strip().replace(".ppm", ".xml")
    curr_et_object.write(filename)
    return curr_et_object

def readAndWrite(bbx_gttxtPath, path_to_img, path_to_data):
    with open(bbx_gttxtPath, 'r') as f:
        prev_img = ''
        path_to_xml = os.path.join(path_to_data, "xmls")
        
        curr_et_object = ET.ElementTree()

        for line in f:
            curr_img, x1, y1, x2, y2,_ = line.split(';')
            
                
            if curr_img!=prev_img:
                img = cv2.imread(os.path.join(path_to_img, curr_img))
                curr_et_object = newXMLPASCALfile(img.shape[0],img.shape[1], path_to_xml, curr_img)
                fileNow = os.path.join(path_to_xml, curr_img)
                prev_img = curr_img
                
            curr_et_object = appendXMLPASCAL(curr_et_object,x1, y1, x2, y2, fileNow)

            
curr_path = os.getcwd()
Train_path = os.path.join(curr_path, "images" )
path_to_data = os.path.join(curr_path, "data", "tf_wider_train")

to_image_folder = os.path.join(path_to_data, "images" )
bbx_gttxtPath = os.path.join(Train_path, 'gt.txt')

try:
    os.makedirs(os.path.join(path_to_data, "xmls" ))
    os.makedirs(to_image_folder)
except Exception as e:
    pass


readAndWrite(bbx_gttxtPath, Train_path, path_to_data)